In [2]:
from openbabel import pybel
from pathlib import Path
from opencadd.structure.core import Structure
import warnings
import subprocess

In [16]:
import nglview as nv

In [22]:
pdb_id = "4DRU"
structure = Structure.from_pdbid(pdb_id)
# element information maybe missing, but important for subsequent PDBQT conversion
if not hasattr(structure.atoms, "elements"):
    structure.add_TopologyAttr("elements", structure.atoms.types)
structure
# NBVAL_CHECK_OUTPUT

<Universe with 9474 atoms>

In [7]:
smiles1 = "Cc1ccc(-n2nncc2-c2cc(-c3ccccc3)sc2C(=O)O)cc1"
smiles2 = "CC(C)N(c1cc(-c2ccccc2)sc1C(=O)O)C(=O)[C@H]1CC[C@H](C)CC1"
smiles3 = "CN(C)CC1COc2ccc3nc4n(c3c2C1)CC(CN(C)C)CN4"
smiles4 = "CN1CCCCCNC(=O)C2=Cc3ccc(Cl)cc3-c3c(C4CCCCC4)c4ccc(cc4n3C2)C(=O)NS1(=O)=O"
smiles5 = "C[C@H]1CC[C@H](n2nncc2-c2cc(-c3ccccc3)sc2C(=O)O)CC1"
smiles6 = "COc1ccc2c(c1)C=C1Cn3c-2c(C2CCCCC2)c2ccc(cc23)C(=O)NS(=O)(=O)N(C)CCOCCN(C)C1=O"
smiles7 = "COc1ccc2c(c1)C(C)=C1Cn3c-2c(C2CCCCC2)c2ccc(cc23)C(=O)NS(=O)(=O)N(C)CCOCCN(C)C1=O"
smiles8 = "COc1ccc2c(c1)C=C1Cn3c-2c(C2CCCCC2)c2ccc(cc23)C(=O)NS(=O)(=O)N(C)CCCCN(C)C1=O"
smiles9 = "COc1ccc2c(c1)C(C)=C1Cn3c-2c(C2CCCCC2)c2ccc(cc23)C(=O)NS(=O)(=O)N(C)CCCCN(C)C1=O"
smiles10 = "CN1CCCCN(C)S(=O)(=O)NC(=O)c2ccc3c(C4CCCCC4)c4n(c3c2)CC(=Cc2ccc(Cl)cc2-4)C1=O"
smiles11 = "CN(C)CCCn1c(N)nc2ccc3c(c21)CC(CN(C)C)O3"
smiles12 = "CN(C)CCCn1c(N)nc2ccccc21"
smiles13 = "COc1ccc2c(c1)C=C1Cn3c-2c(C2CCCCC2)c2ccc(cc23)C(=O)NS(=O)(=O)N(C)CCCCNC1=O"
smiles14 = "COc1ccc2c(c1)C=C1Cn3c-2c(C2CCCCC2)c2ccc(cc23)C(=O)NS(=O)(=O)NCCCCN(C)C1=O"
smiles15 = "COc1ccc2c(c1)C=C1Cn3c-2c(C2CCCCC2)c2ccc(cc23)C(=O)NS(=O)(=O)N(C(C)C)CCCCN(C)C1=O"

In [29]:
ligand_resname1 = "chemBL1210272"
ligand_resname2 = "chemBL561360"
ligand_resname3 = "chemBL223720"
ligand_resname4 = "chemBL3909178"
ligand_resname5 = "chemBL1210273"
ligand_resname6 = "chemBL2043025"
ligand_resname7 = "chemBL3957774"
ligand_resname8 = "chemBL2043027"
ligand_resname9 = "chemBL3983049"
ligand_resname10 = "chemBL2043035"
ligand_resname11 = "chemBL388502"
ligand_resname12 = "chemBL427101"
ligand_resname13 = "chemBL2043036"
ligand_resname14 = "chemBL2043037"
ligand_resname15 = "chemBL2043039"


In [4]:
def smiles_to_pdbqt(smiles, pdbqt_path, pH=7.4):
    """
    Convert a SMILES string to a PDBQT file needed by docking programs of the AutoDock family.

    Parameters
    ----------
    smiles: str
        SMILES string.
    pdbqt_path: str or pathlib.path
        Path to output PDBQT file.
    pH: float
        Protonation at given pH.
    """
    molecule = pybel.readstring("smi", smiles)
    # add hydrogens at given pH
    molecule.OBMol.CorrectForPH(pH)
    molecule.addh()
    # generate 3D coordinates
    molecule.make3D(forcefield="mmff94s", steps=10000)
    # add partial charges to each atom
    for atom in molecule.atoms:
        atom.OBAtom.GetPartialCharge()
    molecule.write("pdbqt", str(pdbqt_path), overwrite=True)
    return

In [5]:
names =locals()
for i in range(1,15):
    HERE = Path(_dh[-1])
    DATA = HERE / str(i)
    smiles_to_pdbqt(names.get('smiles'+str(i)),DATA/"ligand.pdbqt")

TypeError: in method 'OBConversion_ReadString', argument 3 of type 'std::string'

In [11]:
HERE = Path(_dh[-1])
DATA = HERE / str(15)
!smina -r {HERE / "protein.pdbqt"} -l {DATA / "ligand.pdbqt"} -o {DATA / "rdocking_poses.sdf"} --autobox_ligand {DATA / "ligand.pdbqt"} --exhaustiveness 24 --num_modes 10

   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: -209841194

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------


In [9]:
for i in range(1,15):
    HERE = Path(_dh[-1])
    DATA = HERE / str(i)
    print("----------------------------------------------"+str(i)+"---------------------------------------------------")
    !smina -r {HERE / "protein.pdbqt"} -l {DATA / "ligand.pdbqt"} -o {DATA / "redocking_poses.sdf"} --autobox_ligand {DATA / "ligand.pdbqt"} --exhaustiveness 24 --num_modes 10

----------------------------------------------1---------------------------------------------------
   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 981443498

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist

Using random seed: 1546479151

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
1       -5.2       0.000      0.000    
2       -5.0       3.598      6.308    
3       -4.9       3.057      4.216    
4       -4.7       0.867      1.204    
5       -4.6       1.702      2.067    
6       -4.6       3.762      9.355    
7       -4.6       4.374      9.283    
8       -4.6       3.809      7.228    
9       -4.6       3.804      6.647    
10      -4.5       3.671      5.764    
Refine time 36.257
Loop time 36.497
----------------------------------------------7---------------------------------------------------
   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (___

----------------------------------------------12---------------------------------------------------
   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: 1558642727

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | di

In [10]:
HERE = Path(_dh[-1])
DATA = HERE / str(15)
!smina -r {HERE / "protein.pdbqt"} -l {DATA / "ligand.pdbqt"} -o {DATA / "redocking_poses.sdf"} --autobox_ligand {DATA / "ligand.pdbqt"} --exhaustiveness 24 --num_modes 10

   _______  _______ _________ _        _______ 
  (  ____ \(       )\__   __/( (    /|(  ___  )
  | (    \/| () () |   ) (   |  \  ( || (   ) |
  | (_____ | || || |   | |   |   \ | || (___) |
  (_____  )| |(_)| |   | |   | (\ \) ||  ___  |
        ) || |   | |   | |   | | \   || (   ) |
  /\____) || )   ( |___) (___| )  \  || )   ( |
  \_______)|/     \|\_______/|/    )_)|/     \|


smina is based off AutoDock Vina. Please cite appropriately.

Weights      Terms
-0.035579    gauss(o=0,_w=0.5,_c=8)
-0.005156    gauss(o=3,_w=2,_c=8)
0.840245     repulsion(o=0,_c=8)
-0.035069    hydrophobic(g=0.5,_b=1.5,_c=8)
-0.587439    non_dir_h_bond(g=-0.7,_b=0,_c=8)
1.923        num_tors_div

Using random seed: -1875912302

0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------

In [1]:
def split_sdf_file(sdf_path):
    """
    Split an SDF file into seperate files for each molecule.
    Each file is named with consecutive numbers.

    Parameters
    ----------
    sdf_path: str or pathlib.Path
        Path to SDF file that should be split.
    """
    sdf_path = Path(sdf_path)
    stem = sdf_path.stem
    parent = sdf_path.parent
    molecules = pybel.readfile("sdf", str(sdf_path))
    for i, molecule in enumerate(molecules, 1):
        molecule.write("sdf", str(parent / f"{stem}_{i}.sdf"), overwrite=True)
    return

In [4]:
for i in range(1,15):
    HERE = Path(_dh[-1])
    DATA = HERE / str(i)
    split_sdf_file(DATA / "redocking_poses.sdf")

In [19]:
for i in range(1,15):
    HERE = Path(_dh[-1])
    DATA = HERE / str(i)
    docking_pose_id = 1
    view = nv.show_structure_file(
        str(DATA / f"docking_poses_{docking_pose_id}.sdf"),
        representations=[{"params": {}, "type": "licorice"}],
    )
    view.add_pdbid(pdb_id)
    view.render_image(trim=True, factor=2);
    print("----------------------------------------------"+str(i)+"---------------------------------------------------")
    view._display_image()

ValueError: you must provide file extension if using file-like object or text content